In [19]:
import numpy as np
import pandas as pd

from astropy import coordinates
from astropy.coordinates import match_coordinates_sky
import astropy.units as u
import astroquery
from astroquery.irsa import Irsa
from astroquery.vizier import Vizier

Irsa.ROW_LIMIT = -1
Vizier.ROW_LIMIT = -1

import matplotlib.pyplot as plt

%matplotlib inline

# Get the data

In [2]:
obj = ["3C 454.3", 343.49062, 16.14821, 4./60.]
# name, ra, dec, radius of cone
obj_name = obj[0]
obj_ra   = obj[1]
obj_dec  = obj[2]
cone_radius  = obj[3]

In [3]:
obj_coord = coordinates.SkyCoord(ra=obj_ra, dec=obj_dec, unit=(u.deg, u.deg), frame="icrs")

In [12]:
data_2mass = Irsa.query_region(obj_coord, catalog="fp_psc", radius=cone_radius * u.deg)
data_wise  = Irsa.query_region(obj_coord, catalog="allwise_p3as_psd", radius=cone_radius * u.deg)
__data_galex = Vizier.query_region(obj_coord, catalog='II/335', radius=cone_radius * u.deg)
data_galex = __data_galex[0]

In [13]:
num_2mass = len(data_2mass)
num_wise  = len(data_wise)
num_galex = len(data_galex)

print("Number of object in (2MASS, WISE, GALEX): ", num_2mass, num_wise, num_galex)

Number of object in (2MASS, WISE, GALEX):  44 218 41


In [17]:
ra_2mass = data_2mass['ra']
dec_2mass = data_2mass['dec']
c_2mass = coordinates.SkyCoord(ra=ra_2mass, dec=dec_2mass, unit=(u.deg, u.deg), frame="icrs")

ra_wise  = data_wise['ra']
dec_wise = data_wise['dec']
c_wise = coordinates.SkyCoord(ra=ra_wise, dec=dec_wise, unit=(u.deg, u.deg), frame="icrs")

ra_galex  = data_galex['RAJ2000']
dec_galex = data_galex['DEJ2000']
c_galex = coordinates.SkyCoord(ra=ra_galex, dec=dec_galex, unit=(u.deg, u.deg), frame="icrs")

In [89]:
# match_coordinates_sky(matchcoord, catalogcoord, nthneighbor=1, storekdtree='kdtree_sky')
# Return

# idx : integer array
#     Indices into catalogcoord to get the matched points for each matchcoord. Shape matches matchcoord.
# sep2d : Angle
#     The on-sky separation between the closest match for each matchcoord and the matchcoord. Shape matches matchcoord.
# dist3d : Quantity
#     The 3D distance between the closest match for each matchcoord and the matchcoord. Shape matches matchcoord. If either matchcoord or catalogcoord don’t have a distance, this is the 3D distance on the unit sphere, rather than a true distance.

idx_2mass_wise, d2d_2mass_wise, dump = match_coordinates_sky(c_2mass, c_wise)
idx_2mass_galex, d2d_2mass_galex, dump = match_coordinates_sky(c_2mass, c_galex)
idx_wise_galex, d2d_wise_galex, dump = match_coordinates_sky(c_wise, c_galex)

In [98]:
# ID of the closest object (ID in WISE cats) to each of 2MASS obj
# shape = 2MASS
idx_2mass_wise

array([  0,   1,   1,   2,   4,   4,   6,  17,  19,  35,  37,  47,  53,
        56,  59,  70,  80,  83,  89,  95, 108, 126, 130, 139, 143, 151,
       155, 160, 165, 169, 172, 177, 178, 176, 176, 182, 186, 187, 189,
       192, 193, 199, 201, 206])

In [95]:
num_match_2mass_wise  = 0
num_match_2mass_galex = 0
num_match_wise_galex  = 0
num_match_all         = 0
sep_min = 2 * u.arcsec # minimum separation in arcsec

In [96]:
match_2mass_wise = []
for i,d in enumerate(d2d_2mass_wise):
    if d < sep_min:
        match_2mass_wise.append(True)
        num_match_2mass_wise += 1
    else:
        match_2mass_wise.append(False)

match_2mass_galex = []
for i,d in enumerate(d2d_2mass_galex):
    if d < sep_min:
        match_2mass_galex.append(True)
        num_match_2mass_galex += 1
    else:
        match_2mass_galex.append(False)
        
match_wise_galex = []
for i,d in enumerate(d2d_wise_galex):
    if d < sep_min:
        match_wise_galex.append(True)
        num_match_wise_galex += 1
    else:
        match_wise_galex.append(False)

In [97]:
print("Coordinate Match (2MASS-WISE, 2MASS-GALEX, WISE-GALEX): ", num_match_2mass_wise, num_match_2mass_galex, num_match_wise_galex)

Coordinate Match (2MASS-WISE, 2MASS-GALEX, WISE-GALEX):  41 15 18
